# IBM Coursera Capstone Week 3 Assignment
## Clustering Neighbourhoods in Toronto
### Oct 19, 2019

### Part 1

#### Import required libraries

In [1]:
import pandas as pd
# import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

#### Scrape the table from Wikipedia and turn it into a dataframe

In [2]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

### Data cleanup
#### 1. Remove Postcode with "Not Assigned" boroughs

In [4]:
df = df[df.Borough != 'Not assigned']
# df.reset_index(inplace=True)
# df.drop('index',axis = 1)
df.head(10)

Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Not assigned
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern

#### 2. If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough. 

In [7]:
for i, row in df.iterrows():
    if df.loc[i, 'Neighbourhood'] == 'Not assigned':
        df.loc[i, 'Neighbourhood'] = df.loc[i, 'Borough']
    else:
        continue
        
df.head(10)

Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Queen's Park
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern

#### 3. Merge neighbourhoods with the same postcode

In [13]:
df_2 = df.groupby(by=['Postcode','Borough']).agg(lambda x: ', '.join(x))
df_2.reset_index(inplace=True)
df_2

Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H        North York   
29       M3J        North York   
..       ...               ...   
73       M6C              York   
74       M6E              York   
75       M6G  Downtown Toronto   
76       M6H      West Toronto   
77       M6J      West Toronto   
78       M6K      West Toronto   
79       M6L        North York   
80       M6M              York   
81       M6N              York   
82       M6P      West Toronto   
83       M6R      West Toronto   
84       M6S      West Toronto   
85       M7A      Queen's Park   
86       M7R       Mississauga   
87       M7Y      East Toronto   
88       M8V         Etobicoke   
89       M8W         Etobicoke   
90       M8X         Etobicoke   
91       M8Y         Etobicoke   
92       M8Z         Etobicoke   
93       M9A         Etobicoke   
94       M9B         Etobicoke   
95       M9C         Etobicoke   
96       M9L        North York   
97       M9M        North York   
98       M9N              York   
99       M9P         Etobicoke   
100      M9R         Etobicoke   
101      M9V         Etobicoke   
102      M9W         Etobicoke   

                                         Neighbourhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
5                                  Scarborough Village  
6          East Birchmount Park, Ionview, Kennedy Park  
7                      Clairlea, Golden Mile, Oakridge  
8      Cliffcrest, Cliffside, Scarborough Village West  
9                          Birch Cliff, Cliffside West  
10   Dorset Park, Scarborough Town Centre, Wexford ...  
11                                   Maryvale, Wexford  
12                                           Agincourt  
13             Clarks Corners, Sullivan, Tam O'Shanter  
14   Agincourt North, L'Amoreaux East, Milliken, St...  
15                                     L'Amoreaux West  
16                                         Upper Rouge  
17                                   Hillcrest Village  
18                        Fairview, Henry Farm, Oriole  
19                                     Bayview Village  
20                            Silver Hills, York Mills  
21                             Newtonbrook, Willowdale  
22                                    Willowdale South  
23                                     York Mills West  
24                                     Willowdale West  
25                                           Parkwoods  
26                                     Don Mills North  
27                    Flemingdon Park, Don Mills South  
28     Bathurst Manor, Downsview North, Wilson Heights  
29                     Northwood Park, York University  
..                                                 ...  
73                                  Humewood-Cedarvale  
74            

#### Number of rows in the cleaned up dataframe

In [14]:
df_2.shape

(103, 3)

### Part 2

#### Fetch the csv

In [15]:
!wget -q -O 'toneighbourhood_location.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [16]:
location_df = pd.read_csv('toneighbourhood_location.csv')
location_df.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

#### Create two empty columns for Lat and Long

In [29]:
location_df.loc[0, 'Postal Code'] == df_2.loc[0, 'Postcode']

df_2['Latitude'] = ''
df_2['Longitude'] = ''

df_2.head()

Postcode      Borough                           Neighbourhood Latitude  \
0      M1B  Scarborough                          Rouge, Malvern            
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union            
2      M1E  Scarborough       Guildwood, Morningside, West Hill            
3      M1G  Scarborough                                  Woburn            
4      M1H  Scarborough                               Cedarbrae            

  Longitude  
0            
1            
2            
3            
4

#### For loop to match Lat Long into df_2

In [31]:
for i, row in location_df.iterrows():
    for j, row in df_2.iterrows():
        if df_2.loc[j, 'Postcode'] == location_df.loc[i, 'Postal Code']:
            df_2['Latitude'][j] = location_df['Latitude'][i]
            df_2['Longitude'][j] = location_df['Longitude'][i]
        else:
            continue
        
df_2.head(10)

Postcode      Borough                                    Neighbourhood  \
0      M1B  Scarborough                                   Rouge, Malvern   
1      M1C  Scarborough           Highland Creek, Rouge Hill, Port Union   
2      M1E  Scarborough                Guildwood, Morningside, West Hill   
3      M1G  Scarborough                                           Woburn   
4      M1H  Scarborough                                        Cedarbrae   
5      M1J  Scarborough                              Scarborough Village   
6      M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park   
7      M1L  Scarborough                  Clairlea, Golden Mile, Oakridge   
8      M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West   
9      M1N  Scarborough                      Birch Cliff, Cliffside West   

  Latitude Longitude  
0  43.8067  -79.1944  
1  43.7845  -79.1605  
2  43.7636  -79.1887  
3   43.771  -79.2169  
4  43.7731  -79.2395  
5  43.7447  -79.2395  
6  43.7279   -79.262  
7  43.7111  -79.2846  
8  43.7163  -79.2395  
9  43.6927  -79.2648